In [1]:
import csv, pandas as pd, numpy as np

In [2]:
def fixer(filename_list):
    
    df1 = pd.read_csv('../data/{0}'.format(filename_list[0]))
    if len(filename_list) > 1:
        for file in filename_list[1:]:
            new_df = pd.read_csv('../data/{0}'.format(file))
            df1 = pd.concat([df1, new_df])
    
    data = df1.reset_index()
    data = data.fillna(0)
    del data['index']
            
    mapper = pd.read_csv("../data/case_data.csv")
    
    del data['Unnamed: 0']
    data.set_index("county")
    
    data2 = pd.melt(data, id_vars=['county'])
    data3 = data2.rename(columns={'variable':'date', 'county':'FIPS', 'value':'cases'})
    data3['FIPS'] = data3['FIPS'].apply(lambda x: str(x) if len(str(x)) == 5 else str(x))
    data4 = data3
    mapper['FIPS'] = mapper['FIPS'].apply(lambda x: str(x).split(".")[0])
    data4 = data4.merge(mapper, on='FIPS')
    
    del data4['Country_Region']
    del data4['Last_Update']
    del data4['Lat']
    del data4['Long_']
    del data4['Confirmed']
    del data4['Deaths']
    del data4['Recovered']
    del data4['Active']
    del data4['Combined_Key']
    
    data5 = data4.sort_values('date')
    data5 = data5.rename(columns={'FIPS':'fips', 'Admin2':'county', 'Province_State':'state'})
    ir = filename_list[0].split("_")[2]
    rr = filename_list[0].split("_")[3]
    
    #get historical data from nyt
    historical_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")
    del historical_data['deaths']
    #print(historical_data['state'].unique())
    #format fips correctly
    #historical_data['fips'] = historical_data['fips'].apply(lambda x: str(x) if len(str(x)) == 5 else "0"+str(x))
    historical_data['fips'] = historical_data['fips'].apply(lambda x: str(x).split(".")[0])
    #only include the fips ids we're looking at for this viz
    historical_data = historical_data.loc[historical_data['fips'].isin(list(data5['fips']))]
    historical_data.reset_index()
    indexNames = historical_data[ historical_data['fips'] == 'nan' ].index
    historical_data.drop(indexNames, inplace=True)
    #filter dates by after a date
    historical_data.sort_values('date')
    historical_data['date'] = pd.to_datetime(historical_data['date'])
    historical_data = historical_data[(historical_data["date"] >= '2020-04-18') & (historical_data["date"] <= '2020-04-21')]
    historical_data['date'] = historical_data['date'].apply(lambda x: str(x).split(" ")[0]) 
    #decrease cases by factor of 1000 in order to match our predicted data
    historical_data['cases'] = historical_data['cases'].apply(lambda x: x/1000)
    #compile together
    data6 = pd.concat([historical_data,data5], ignore_index=True)
    data6.sort_values(by=['date'])
    data6['fips'] = data6['fips'].apply(lambda x: "0" + x if len(x) < 5 else x)
    
    data6.to_csv("../data/combined_{0}_{1}".format(ir, rr))

In [3]:
tester_full = ["output_Connecticut_0.001_0.06.csv","output_Maine_0.001_0.06.csv", "output_Vermont_0.001_0.06.csv", "output_New Hampshire_0.001_0.06.csv", "output_Rhode Island_0.001_0.06.csv"]
tester_mini = ["output_Maine_0.001_0.06.csv", "output_Vermont_0.001_0.06.csv"]
fixer(tester_full)

/Users/Elise/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/Users/Elise/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

